# KRX 분기 재무제표 데이터 수집

In [1]:
import numpy as np
import pandas as pd
import nfn_crawring as nf
import gbq_service as gbq

In [2]:
df_krx = gbq.get_krx_list()
df_krx = df_krx.sort_values('market_cap', ascending=False).reset_index(drop=True)
df_krx

,stock_code,corp_name,ipo_date,marker_type,dept,stock_num,market_cap
0,005930,삼성전자,1975-06-11,KOSPI,None,5969782550,444151821720000
1,000660,SK하이닉스,1996-12-26,KOSPI,None,728002365,98280319275000
2,373220,LG에너지솔루션,2022-01-27,KOSPI,None,234000000,88335000000000
3,207940,삼성바이오로직스,2016-11-10,KOSPI,None,71174000,56939200000000
4,005380,현대차,1974-06-28,KOSPI,None,211531506,41375562573600
...,...,...,...,...,...,...,...
2325,030790,비케이탑스,2010-11-02,KOSPI,None,21248145,19208323080
2326,071950,코아스,2005-08-04,KOSPI,None,30907210,15824491520
2327,078860,아이오케이,2006-06-27,KOSDAQ,중견기업부,4806323,15788771055
2328,008500,일정실업,1994-08-12,KOSPI,None,1200000,15480000000


In [3]:
# collection_list = df_krx[['stock_code', 'corp_name']].values[0:10]
collection_list = df_krx[['stock_code', 'corp_name']].values

In [4]:
df_gbq = pd.DataFrame()
df_exception = pd.DataFrame(columns=['stock_code', 'corp_name', 'err_msg'])

for index, corp in enumerate(collection_list):
    try:
        print('No. ', index)
        print(corp, '재무제표 데이터 크롤링 시작 ===>')
        # 분기 손익계산서
        df_is = nf.crawl_nfn(stock_code=corp[0], type=nf.DataType.FS, subType=nf.FSType.IS, period=nf.Period.QUARTER)
        df_is = gbq.convert_gbq_quarter(df_is, corp[0], corp[1], 'IS')
        
        # 분기 재무상태표
        df_bs = nf.crawl_nfn(stock_code=corp[0], type=nf.DataType.FS, subType=nf.FSType.BS, period=nf.Period.QUARTER)
        df_bs = gbq.convert_gbq_quarter(df_bs, corp[0], corp[1], 'BS')
        
        # 분기 현금흐름표
        df_cf = nf.crawl_nfn(stock_code=corp[0], type=nf.DataType.FS, subType=nf.FSType.CF, period=nf.Period.QUARTER)
        df_cf = gbq.convert_gbq_quarter(df_cf, corp[0], corp[1], 'CF')
        
        # 데이터 합치기
        df_gbq = pd.concat([df_gbq, df_is, df_bs, df_cf])
        
        print(corp, '재무제표 데이터 크롤링 종료 ===>\n\n')
        
    except Exception as e:
        err_msg = str(type(e)) + ': ' + str(e)
        print(err_msg)
        df_exception.loc[len(df_exception)] = [corp[0], corp[1], err_msg]

No.  0
['005930' '삼성전자'] 재무제표 데이터 크롤링 시작 ===>
https://navercomp.wisereport.co.kr/v2/company/cF3002.aspx?cmp_cd=005930&frq=1&rpt=0&finGubun=MAIN&frqTyp=1&cn=&encparam=QWdiampJbUFQczNONjFxS2RSL05Ndz09
https://navercomp.wisereport.co.kr/v2/company/cF3002.aspx?cmp_cd=005930&frq=1&rpt=1&finGubun=MAIN&frqTyp=1&cn=&encparam=TlBQY1ZhdXVLZGhkaWx1bVo2dnZlZz09
https://navercomp.wisereport.co.kr/v2/company/cF3002.aspx?cmp_cd=005930&frq=1&rpt=2&finGubun=MAIN&frqTyp=1&cn=&encparam=ZUgwNzFhak1aM3BINTlPRWhsQi9Odz09
['005930' '삼성전자'] 재무제표 데이터 크롤링 종료 ===>


No.  1
['000660' 'SK하이닉스'] 재무제표 데이터 크롤링 시작 ===>
https://navercomp.wisereport.co.kr/v2/company/cF3002.aspx?cmp_cd=000660&frq=1&rpt=0&finGubun=MAIN&frqTyp=1&cn=&encparam=ZWtscURqVXcyS01QNUJ5TGxsZ09DUT09
https://navercomp.wisereport.co.kr/v2/company/cF3002.aspx?cmp_cd=000660&frq=1&rpt=1&finGubun=MAIN&frqTyp=1&cn=&encparam=UWRiMVNQTmlrS1ZxZUFMc1lheGxmUT09
https://navercomp.wisereport.co.kr/v2/company/cF3002.aspx?cmp_cd=000660&frq=1&rpt=2&finGubun=MAIN&fr

In [5]:
df_gbq.to_csv('fs_quarter_240201.csv')

In [6]:
df_exception

,stock_code,corp_name,err_msg
0,068400,SK렌터카,<class 'requests.exceptions.JSONDecodeError'>:...
